I want to predict if an NBA team will make the playoffs. I chose this for my first Machine Learning project because I already know which variables increase the chances of making the playoffs. I am also initerested in the power of the variables when creating the models.

In [1]:
import matplotlib.pyplot as plt  # To visualize
import matplotlib
matplotlib.use("Qt5Agg")
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import export_graphviz
from sklearn import tree
import numpy as np
from sklearn import metrics
pd.options.mode.chained_assignment = None  # default='warn'

First, I am going to initialize some lists and variables that I will use for iteration purposes or throughout the program

In [185]:
# Variables I'll need throughout
path = "/Users/matthewmacrides/Python and R/Github/Machine Learning/NbaPlayoffs"

sep = r'/'

seasons = ['18-19', '21-22']

models = ['Decision Tree', 'Random Forest']

stats = ['PS','Wins','PA','PD','FG%', '3P%', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF']

I decided to make a for-loop in the case that I add more years of data in the future. All I will need to do is save the .csv file and update the seasons list.

In [186]:
def readData():
     # Create dataframes that will store all the years data
     Nba = pd.DataFrame()

     for season in seasons:
          Nba_ = pd.read_csv(path+sep+season+'.csv', sep=',', decimal='.')
          
          # Create a season column at the end of each file
          Nba_['Season'] = season
          
          # Concatenate files together for each category
          Nba = pd.concat([Nba,Nba_], ignore_index=True)

          Nba.dropna(how='any', inplace=True)

          Teams = Nba[['Team', 'Season']].copy()
          
     return Nba, Teams

I was interested in the correlations across variables. To no surprise, Wins, Losses, Win Percentage, PD, and Standardized PD had strong correlations to Playoffs. I am going to include these variables in my first iteration of the modeliing. Then I am going to remove them to capture the change in accuracy. I am assuming my models will have an accuracy score close to 1 during my first iteration since the variables I mentioned are directly embedded into a team's playoff chances.

In [192]:
Nba, Teams = readData()
Nba.corr().style.background_gradient(cmap="Blues")

,Seed,Wins,Losses,Win Percentage,PS,PA,PD,Standardized PD,FG%,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,Year,Playoff,QF,SF,C
Seed,1.000000,-0.957603,0.957603,-0.957713,-0.579814,0.601987,-0.922434,-0.920029,-0.330752,-0.346985,-0.306488,-0.441456,-0.394662,-0.227805,-0.157820,0.286439,0.131091,-0.000000,-0.850561,-0.767649,-0.533357,-0.236391
Wins,-0.957603,1.000000,-1.000000,0.999998,0.626843,-0.607624,0.963498,0.960986,0.308705,0.412252,0.255449,0.459743,0.432330,0.220209,0.201213,-0.324080,-0.136078,0.000000,0.832188,0.682198,0.460635,0.232085
Losses,0.957603,-1.000000,1.000000,-0.999998,-0.626843,0.607624,-0.963498,-0.960986,-0.308705,-0.412252,-0.255449,-0.459743,-0.432330,-0.220209,-0.201213,0.324080,0.136078,-0.000000,-0.832188,-0.682198,-0.460635,-0.232085
Win Percentage,-0.957713,0.999998,-0.999998,1.000000,0.627022,-0.607553,0.963583,0.961079,0.308621,0.412242,0.255040,0.460130,0.432439,0.220368,0.201503,-0.323994,-0.136303,-0.000118,0.832284,0.682190,0.460361,0.231731
PS,-0.579814,0.626843,-0.626843,0.627022,1.000000,0.179255,0.639392,0.643546,0.273902,0.405337,0.098689,0.493603,0.565875,0.264852,0.337408,-0.022243,0.089380,-0.079888,0.398450,0.395822,0.242361,0.089257
PA,0.601987,-0.607624,0.607624,-0.607553,0.179255,1.000000,-0.641813,-0.634269,-0.064329,-0.129820,-0.160919,-0.156099,-0.023907,-0.013820,-0.021394,0.369391,0.311979,-0.080568,-0.599348,-0.422631,-0.361321,-0.196478
PD,-0.922434,0.963498,-0.963498,0.963583,0.639392,-0.641813,1.000000,0.997342,0.263779,0.417414,0.202691,0.506755,0.459776,0.217250,0.279725,-0.306032,-0.174153,0.000696,0.779001,0.638840,0.471304,0.223131
Standardized PD,-0.920029,0.960986,-0.960986,0.961079,0.643546,-0.634269,0.997342,1.000000,0.273514,0.408799,0.192276,0.519573,0.457100,0.216073,0.290727,-0.313465,-0.139860,-0.072171,0.776970,0.637175,0.470075,0.222549
FG%,-0.330752,0.308705,-0.308705,0.308621,0.273902,-0.064329,0.263779,0.273514,1.000000,-0.449159,0.263556,0.037558,0.383117,-0.017983,0.097278,-0.028515,-0.009798,-0.143051,0.277632,0.314114,0.248379,0.118633
3P%,-0.346985,0.412252,-0.412252,0.412242,0.405337,-0.129820,0.417414,0.408799,-0.449159,1.000000,0.128708,0.114698,0.201550,0.010749,0.002822,-0.181420,-0.155778,0.103298,0.312974,0.090442,0.091942,0.050906


Saving a copy of the Team column so we can join with our test data after the analysis. Also creating a copy of the Nba raw data file in the case that I model whether or not a team will make the quarterfinals. Lastly, I remove all string columns and additional columns that aren't important for this analysis


In [193]:
Nba = Nba.drop(columns=['Team', 'Year', 'QF', 'SF', 'F', 'C'])

Here, I define a function called 'modeling' to split test and training data. I will chose Playoff as my y value. Playoff will be denoted as a '1' if the team made it and a '0' otherwise. Then I iterated through different models and recorded the accuracy for all of them.

In [194]:
def modeling(models, Nba):
    # Create x and y datasets with the target variable 'Playoff' in mind. 0 denotes a team does not make the playoffs. 1 denotes that a team does make the playoffs.
    y = Nba[["Playoff", "Season"]]
    x = Nba.drop('Playoff', axis=1)
    for season in seasons:

        # Separating training and testing data
        season_test = season

        x_train = x[x['Season']!=season_test]
        x_test = x[x['Season']==season_test]
        y_train = y[y['Season']!=season_test]
        y_test = y[y['Season']==season_test]

    datasets = [x_train, x_test, y_train, y_test]
    for dataset in datasets:
        dataset.drop(['Season'], axis=1, inplace=True)

    feature_list = list(x_train.columns)
    
    metric = pd.DataFrame()
    compare = pd.DataFrame()
    features = pd.DataFrame()
    
    for model in models:
            if model == 'Decision Tree':
                model_ = DecisionTreeClassifier() 
            elif model=='Random Forest':
                model_ = RandomForestClassifier()
            model_.fit(x_train, y_train.values.ravel())
            
            y_pred = model_.predict(x_test)
            # Store the accuracy in a dataframe. Concatenate for each accuracy score and record the model that received the result
            accuracy = metrics.accuracy_score(y_test, y_pred)
            data = {'Accuracy Score': [accuracy], 'Model': [model]}
            accuracy = pd.DataFrame(data)
            metric = pd.concat([metric,accuracy], ignore_index=True)

            # Comparing real vs. expected playoffs
            compare['Playoff'] = y_test
            compare['Pred Playoff '+ model] = y_pred

            # Store the variable importances
            # Get numerical feature importances
            importances = list(model_.feature_importances_)
            # List of tuples with variable and importance
            feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
            # Sort the feature importances by most important first
            feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
            features_ = pd.DataFrame(feature_importances, columns=['Variables', 'Importance'])
            features_['Model'] = model
            features_ = features_.head(5)
            features = pd.concat([features, features_], ignore_index=True)
            
    return metric, compare, features

As we can see, the models are almost 100% accurate when predicting the playoff teams. The only reason they aren't 100% is because there are years that non-playoff teams in the west have a better record than playoff teams in the east and vice versa.

In [195]:
metric, compare, features = modeling(models, Nba)
metric

,Accuracy Score,Model
0,0.933333,Decision Tree
1,0.933333,Random Forest


In [196]:
features

,Variables,Importance,Model
0,Wins,1.00,Decision Tree
1,Seed,0.00,Decision Tree
2,Losses,0.00,Decision Tree
3,Win Percentage,0.00,Decision Tree
4,PS,0.00,Decision Tree
5,Win Percentage,0.22,Random Forest
6,Wins,0.18,Random Forest
7,Losses,0.18,Random Forest
8,Seed,0.14,Random Forest
9,PD,0.12,Random Forest


Thiis is just showing the team, if they made the playoffs, and what the models predicted.

In [197]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Pred Playoff Random Forest
30,Phoenix Suns,21-22,1.0,1.0,1.0
31,Memphis Grizzlies,21-22,1.0,1.0,1.0
32,Golden State Warriors,21-22,1.0,1.0,1.0
33,Miami Heat,21-22,1.0,1.0,1.0
34,Dallas Mavericks,21-22,1.0,1.0,1.0
35,Boston Celtics,21-22,1.0,1.0,1.0
36,Milwaukee Bucks,21-22,1.0,1.0,1.0
37,Philadelphia 76ers,21-22,1.0,1.0,1.0
38,Utah Jazz,21-22,1.0,1.0,1.0
39,Denver Nuggets,21-22,1.0,1.0,1.0


In [198]:
NbaSmall = Nba.drop(columns=['Wins', 'Losses', 'Win Percentage', 'PS', 'PA', 'PD', 'Standardized PD'])

In [199]:
metric, compare, features = modeling(models, NbaSmall)
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
metric

,Accuracy Score,Model
0,0.866667,Decision Tree
1,0.866667,Random Forest


In [200]:
features

,Variables,Importance,Model
0,Seed,1.00,Decision Tree
1,FG%,0.00,Decision Tree
2,3P%,0.00,Decision Tree
3,FT%,0.00,Decision Tree
4,TRB,0.00,Decision Tree
5,Seed,0.48,Random Forest
6,3P%,0.20,Random Forest
7,FG%,0.07,Random Forest
8,FT%,0.05,Random Forest
9,BLK,0.05,Random Forest


In [201]:
compare

,Team,Season,Playoff,Pred Playoff Decision Tree,Pred Playoff Random Forest
30,Phoenix Suns,21-22,1.0,1.0,1.0
31,Memphis Grizzlies,21-22,1.0,1.0,1.0
32,Golden State Warriors,21-22,1.0,1.0,1.0
33,Miami Heat,21-22,1.0,1.0,1.0
34,Dallas Mavericks,21-22,1.0,1.0,1.0
35,Boston Celtics,21-22,1.0,1.0,1.0
36,Milwaukee Bucks,21-22,1.0,1.0,1.0
37,Philadelphia 76ers,21-22,1.0,1.0,1.0
38,Utah Jazz,21-22,1.0,1.0,1.0
39,Denver Nuggets,21-22,1.0,1.0,1.0


In [36]:
import pydot
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=10, max_depth = 3)
rf_small.fit(x_train, y_train.values.ravel())
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [203]:
Nba, Teams = readData()
NbaQF = Nba[Nba['Playoff'] == 1]
NbaQF = NbaQF.drop(columns=['Team', 'Year', 'Playoff', 'SF', 'F', 'C'])
NbaQF = NbaQF.drop(columns=['Wins', 'Losses', 'Win Percentage', 'PS', 'PA', 'PD', 'Standardized PD'])

In [204]:
def modelingQF(models, NbaQF):
    # Create x and y datasets with the target variable 'Playoff' in mind. 0 denotes a team does not make the playoffs. 1 denotes that a team does make the playoffs.
    y = NbaQF[["QF", "Season"]]
    x = NbaQF.drop('QF', axis=1)
    for season in seasons:

        # Separating training and testing data
        season_test = season

        x_train = x[x['Season']!=season_test]
        x_test = x[x['Season']==season_test]
        y_train = y[y['Season']!=season_test]
        y_test = y[y['Season']==season_test]

    datasets = [x_train, x_test, y_train, y_test]
    for dataset in datasets:
        dataset.drop(['Season'], axis=1, inplace=True)

    feature_list = list(x_train.columns)
    
    metric = pd.DataFrame()
    compare = pd.DataFrame()
    features = pd.DataFrame()
    
    for model in models:
            if model == 'Decision Tree':
                model_ = DecisionTreeClassifier() 
            elif model=='Random Forest':
                model_ = RandomForestClassifier()
            model_.fit(x_train, y_train.values.ravel())
            
            y_pred = model_.predict(x_test)
            # Store the accuracy in a dataframe. Concatenate for each accuracy score and record the model that received the result
            accuracy = metrics.accuracy_score(y_test, y_pred)
            data = {'Accuracy Score': [accuracy], 'Model': [model]}
            accuracy = pd.DataFrame(data)
            metric = pd.concat([metric,accuracy], ignore_index=True)

            # Find probabilities
            probability = model_.predict_proba(x_test)
            probability = probability[:, 1]
            #probability_ = pd.DataFrame(probability_, columns=['Probability'])
            #probability = pd.concat([probability, probability_], ignore_index=True)

            # Comparing real vs. expected playoffs
            compare['QF'] = y_test
            compare['Pred Playoff ' + model] = y_pred
            compare['Probability ' + model] = probability

            # Store the variable importances
            # Get numerical feature importances
            importances = list(model_.feature_importances_)
            # List of tuples with variable and importance
            feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
            # Sort the feature importances by most important first
            feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
            features_ = pd.DataFrame(feature_importances, columns=['Variables', 'Importance'])
            features_['Model'] = model
            features_ = features_.head(5)
            features = pd.concat([features, features_], ignore_index=True)
            
    return metric, compare, features

In [205]:
metric, compare, features = modelingQF(models, NbaQF)

In [206]:
metric

,Accuracy Score,Model
0,1.0,Decision Tree
1,1.0,Random Forest


In [207]:
compare = pd.merge(Teams, compare, left_index=True, right_index=True)
compare

,Team,Season,QF,Pred Playoff Decision Tree,Probability Decision Tree,Pred Playoff Random Forest,Probability Random Forest
30,Phoenix Suns,21-22,1.0,1.0,1.0,1.0,0.90
31,Memphis Grizzlies,21-22,1.0,1.0,1.0,1.0,0.62
32,Golden State Warriors,21-22,1.0,1.0,1.0,1.0,0.79
33,Miami Heat,21-22,1.0,1.0,1.0,1.0,0.73
34,Dallas Mavericks,21-22,1.0,1.0,1.0,1.0,0.59
35,Boston Celtics,21-22,1.0,1.0,1.0,1.0,0.73
36,Milwaukee Bucks,21-22,1.0,1.0,1.0,1.0,0.65
37,Philadelphia 76ers,21-22,1.0,1.0,1.0,1.0,0.76
38,Utah Jazz,21-22,0.0,0.0,0.0,0.0,0.24
39,Denver Nuggets,21-22,0.0,0.0,0.0,0.0,0.42


In [208]:
features

,Variables,Importance,Model
0,Seed,1.00,Decision Tree
1,FG%,0.00,Decision Tree
2,3P%,0.00,Decision Tree
3,FT%,0.00,Decision Tree
4,TRB,0.00,Decision Tree
5,Seed,0.37,Random Forest
6,FT%,0.14,Random Forest
7,AST,0.09,Random Forest
8,STL,0.08,Random Forest
9,PF,0.08,Random Forest
